In [3]:
#importing libraries for voice recorder
from re import X
import tkinter as tk
from tkinter import messagebox
import threading
from numpy import imag
import pyaudio
from tkinter import SUNKEN, Button, Frame, PhotoImage,Label
from PIL import Image, ImageTk
import wave

#importing libraries for speech to text conversion
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
import speech_recognition as sr

#importing libraries for gif
from itertools import count

#importing libraries to load model
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
from joblib import Parallel, delayed
import joblib

# Load the model from the file
obj = joblib.load('saved_model.pkl')
model=obj[0]
vectorizer=obj[1]
#mh_data=pd.read_csv('stemmed_data.csv')
#X=mh_data['text'].values
#Y=mh_data['class'].values
#vectorizer=TfidfVectorizer()
#vectorizer.fit(X)

#recording audio on interface
class App():
    chunk = 1024 
    sample_format = pyaudio.paInt16 
    channels = 2
    fs = 44100  
    
    frames = []  
    images_list=[]
    gif_duration=0
    xy=0
    global pred
    
    def extract_images(self,path):
        global gif_duration
        image=Image.open(path)
        for r in count(1):
            try:
                self.images_list.append(image.copy())
                image.seek(r)
            except Exception as e:
                break
        print(len(self.images_list))
        gif_duration=int(image.info['duration'])

    def play_gif(self):
        global xy,cur_img
        try:
            xy+=1
            resize_img=self.images_list[xy].resize(
                (900,200),Image.ANTIALIAS
            )
            cur_img=ImageTk.PhotoImage(resize_img)
            self.gif_lb.configure(image=cur_img)
            main.after(gif_duration,self.play_gif)
        except Exception as e:
            xy=0
            main.after(gif_duration,self.play_gif)
        

    def __init__(self, master):
        self.isrecording = False
        self.button1 = tk.Button(main,font="arial 12 bold",text="RECORD",bg="#7ED9ED",activebackground="blue",command=self.startrecording).place(x=300,y=280)
        self.button2 = tk.Button(main,font="arial 12 bold",text="STOP",bg="#7ED9ED",activebackground="blue",command=self.stoprecording).place(x=450,y=280)
        self.gif_lb=tk.Label(main)
        self.gif_lb.place(x=0,y=50)

    def startrecording(self):
        global xy
        xy=0
        self.p = pyaudio.PyAudio()  
        self.stream = self.p.open(format=self.sample_format,channels=self.channels,rate=self.fs,frames_per_buffer=self.chunk,input=True)
        self.isrecording = True
        
        print('Recording')
        t = threading.Thread(target=self.record)
        t.start()
        
        self.extract_images("sound_wave.gif")
        self.play_gif()

        self.label3=tk.Label(main,text="RECORDING AUDIO...",font="arial 10 bold",fg="red").place(x=300,y=315)

    def stoprecording(self):
        #stemming
        port_stem=PorterStemmer()
        def stemming(content):
            stemmed_content=re.sub('[^a-zA-Z]',' ',content)
            stemmed_content=stemmed_content.lower()
            stemmed_content=stemmed_content.split()
            stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
            stemmed_content=' '.join(stemmed_content)
            return stemmed_content
        
        self.isrecording = False
        print('recording complete')
        self.filename='recording'
        self.filename = self.filename+".wav"
        wf = wave.open(self.filename, 'wb')
        wf.setnchannels(self.channels)
        wf.setsampwidth(self.p.get_sample_size(self.sample_format))
        wf.setframerate(self.fs)
        wf.writeframes(b''.join(self.frames))
        wf.close()
        #-------------------speech to text conversion----------------------------------------------------------
        r=sr.Recognizer()
        path="recording.wav"
        data=self.get_large_audio_transcription(path,r)
        self.label=tk.Label(frame1,text=data,font="arial 15 bold",fg="blue",bg="#7ED9ED").place(x=5,y=5)
        self.gif_lb.destroy()
        self.label3=tk.Label(main,text="RECORDING COMPLETE",font="arial 10 bold",fg="green").place(x=300,y=315)
        #-------------------detection---------------------------------------------------------------------------
        data=pd.Series(data)
        data=data.apply(stemming)
        data=data.values
        #vect=TfidfVectorizer()
        #vect.fit(data)
        data=vectorizer.transform(data)
        data=csr_matrix(data)
        #csr_matrix.resize(data,(1,116181))
        pred=model.predict(data)
        messagebox.showinfo("Predicted result",pred)
        #self.button3 = tk.Button(main,font="arial 12 bold",text="PREDICT RESULT",bg="#7ED9ED",activebackground="blue",command=self.popup).place(x=550,y=280)
        
    def popup(self):
        messagebox.showinfo("Predicted Result",pred)
    
    def get_large_audio_transcription(self,path,r):
        sound = AudioSegment.from_wav(path)  
        chunks = split_on_silence(sound,min_silence_len = 500,silence_thresh = sound.dBFS-14,keep_silence=500,)
        folder_name = "audio-chunks"
        if not os.path.isdir(folder_name):
            os.mkdir(folder_name)
        whole_text = ""
        for i, audio_chunk in enumerate(chunks, start=1):
            chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
            audio_chunk.export(chunk_filename, format="wav")
            with sr.AudioFile(chunk_filename) as source:
                audio_listened = r.record(source)
                try:
                    text = r.recognize_google(audio_listened)
                except sr.UnknownValueError as e:
                    print("Error:", str(e))
                else:
                    text = f"{text.capitalize()}. "+"\n"
                    print(chunk_filename, ":", text)
                    whole_text += text
        return whole_text

    def record(self):
       
        while self.isrecording:
            data = self.stream.read(self.chunk)
            self.frames.append(data)
		
main=tk.Tk()
main.title("Suicidal Thoughts Detection in Mentally Disturbed person (ML)")
main.geometry('900x700')
main.resizable(False,False)

#icon
#main.iconphoto(False,PhotoImage(file="logo.png"))

#LAbel 1
fl=Label(main,text="speak out your mind",relief="ridge" ,font="arial 15 bold",bg="blue",fg="white").place(x=300,y=5)
#Label 2
names=tk.Label(main,font="arial 12 bold",text="Made By- Harshada, Samrudhhi, Vrushali, Srushti",fg="blue").place(x=260,y=670)


#Recorder image
image=Image.open("wavelength.png")
resize_image=image.resize((900,200))
img=ImageTk.PhotoImage(resize_image)
label2=Label(image=img)
label2.image=img
label2.place(x=0,y=50)

#Creating frames
frame1=Frame(main,width=890,height=300,bg="#7ED9ED",bd=5,relief=SUNKEN)
frame1.place(x=5,y=350)

app = App(main)
main.mainloop()

Recording
10
recording complete
Error: 
audio-chunks\chunk2.wav : Need help remembering and name of an old toy and member when i was younger bad in 2012. 

audio-chunks\chunk3.wav : Remember that kids around me use plastic ball to the magnet on the bottom. 

